In [198]:
import os
import numpy as np
import pandas as pd

from Bio.Seq import Seq
from Bio import SeqIO
#from Bio.Alphabet import Alphabet
from Bio.SeqRecord import SeqRecord

pd.options.mode.chained_assignment = None
import time

DIR = r'D:\Dropbox\Sandra Postdoc\Experiments\COVID-19 panning\NGS data Pannings\Results\RpMs\441'

In [199]:
list_sequences = []
rpm = []

filename="441_elu77_NNK1_5_unique_rpm"

with open(os.path.join(DIR, filename+'.faa'), 'r') as f:  

        n=0
        m=1

        for i, line in enumerate(f):
            if i == n:
                rpm.append(line[26:-1])
                n+=2             
            if i == m:
                list_sequences.append(line[:-1])
                m+=2
            #if i>10002:
              #  break
            
print(len(list_sequences))
print(len(rpm))

df_read = pd.DataFrame(
    {'sequence_read': list_sequences,
    'rpm': rpm
    })
df_read.head(20)

98705
98705


,sequence_read,rpm
0,ELWSNNLDSKVGGNYNYLYRLFQLYHYQAGSTPCNGVEGFNCYF,s_517013.12672738207
1,ELWSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCYFP,s_92253.43886252563
2,ELWWNSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCY,s_25353.794139790505
3,ELWSNNLDSKVGGNYNYLYRLFAPDGAYQAGSTPCNGVEGFNCY,s_22586.92489148723
4,ELWNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCYFPLQ,s_15976.162357245388
5,ELWDSKVGGNYNYLYRLFQLENYRAGSTPCNGVEGFNCYFPLQS,s_12877.195407388472
6,ELWSNNLDSKVGGNYNYLYRLFRLPVYQAGSTPCNGVEGFNCYF,s_9930.516353824109
7,ELWSNNLDSKVGGNYNYLYRLFQLYHYQAGSTPCNGVEGFKCYF,s_7237.038862770258
8,ELWNSNNLDSKVGGNYNYLYRLFLLMKYQAGSTPCNGVEGFNCY,s_7231.534480976551
9,ELWWNSNNLDSKVGGNYNYLYRLFQLYHYQAGSTPCNGVEGFNC,ts_7042.5507060592845


In [200]:
df_read['rpm']=df_read['rpm'].str.strip('counts_')
df_read

,sequence_read,rpm
0,ELWSNNLDSKVGGNYNYLYRLFQLYHYQAGSTPCNGVEGFNCYF,517013.12672738207
1,ELWSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCYFP,92253.43886252563
2,ELWWNSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCY,25353.794139790505
3,ELWSNNLDSKVGGNYNYLYRLFAPDGAYQAGSTPCNGVEGFNCY,22586.92489148723
4,ELWNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCYFPLQ,15976.162357245388
...,...,...
98700,_LWWNSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGVNCY,0.611597977078531
98701,_LWWNSNNLDSKVGGNYNYLYRLFR_RYQAGSTQCNGVEGFNCN,0.611597977078531
98702,_LWWNSNNLDSKVGGNYNYLYRLFVRTEYQAGSTPCNGVEGFNC,0.611597977078531
98703,_LWWNSNNLDSKVGGNYNYLYRLIRLRYQAGSTPCNGVEGFNCY,0.611597977078531


In [201]:
print(df_read['rpm'].value_counts())

0.611597977078531     83830
1.223195954157062      5948
1.834793931235593      2517
2.446391908314124      1318
3.0579898853926553      834
                      ...  
128.43557518649152        1
222.01006567950677        1
533.9250339895576         1
394.48069521565253        1
1733.268667040557         1
Name: rpm, Length: 389, dtype: int64


In [202]:
import regex

def before_linker(sequence):
    matches = regex.search('(NYNYLYRLF){e<=1}', sequence, regex.BESTMATCH)
    if matches ==None:
        return ('no match')
    else:
        index_end = matches.end()
        if sequence[index_end-1] == 'F':
            return (index_end)
        if sequence[index_end-1] == 'V':
            return (index_end)
        else:
            return ('no F')
        
def after_linker(sequence):
    matches = regex.search('(YQAGSTP){e<=1}', sequence, regex.BESTMATCH)
    if matches ==None:
        return ('no match')
    else:
        index_start = matches.start()
        if sequence[index_start] == 'Y':
            return (index_start)
        else:
            return ('no Y')
        
        
df_read['start_linker'] = df_read['sequence_read'].apply(before_linker)
df_read['end_linker'] = df_read['sequence_read'].apply(after_linker)

df_read
print(df_read['start_linker'].value_counts())
print(df_read['end_linker'].value_counts())

22          67473
no match    11372
20           3963
18           3922
24           3224
no F         3120
23           2195
21           1872
19           1161
25            389
17              6
26              4
27              3
28              1
Name: start_linker, dtype: int64
26          45170
no match    17660
25          15038
27           6391
23           3466
no Y         3261
22           3194
24           1930
28           1499
21            622
29            347
20             39
19             23
30             19
18             15
14             11
16              8
13              5
15              3
32              2
31              1
17              1
Name: end_linker, dtype: int64


In [203]:
#determine start and end of peptide and remove sequences that don't fit filter

df_read_integers= df_read[df_read.start_linker.apply(lambda x: isinstance(x,int))]
df_read_integers= df_read_integers[df_read_integers.end_linker.apply(lambda x: isinstance(x,int))]

df_read_integers

,sequence_read,rpm,start_linker,end_linker
0,ELWSNNLDSKVGGNYNYLYRLFQLYHYQAGSTPCNGVEGFNCYF,517013.12672738207,22,26
1,ELWSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCYFP,92253.43886252563,22,25
2,ELWWNSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCY,25353.794139790505,24,27
3,ELWSNNLDSKVGGNYNYLYRLFAPDGAYQAGSTPCNGVEGFNCY,22586.92489148723,22,27
4,ELWNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGFNCYFPLQ,15976.162357245388,20,23
...,...,...,...,...
98698,_LWWNSNNLDSKVGGNYNYLYRLFRLPIYQAGSTPCNGVEGFNC,0.611597977078531,24,28
98699,_LWWNSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEEFNCY,0.611597977078531,24,27
98700,_LWWNSNNLDSKVGGNYNYLYRLFRLRYQAGSTPCNGVEGVNCY,0.611597977078531,24,27
98701,_LWWNSNNLDSKVGGNYNYLYRLFR_RYQAGSTQCNGVEGFNCN,0.611597977078531,24,27


In [204]:
df_read_integers['sequence_read'] = df_read_integers.apply(lambda x: x['sequence_read'][x['start_linker']:x['end_linker']], 1)  

df_read_integers
df_read_integers['rpm']=df_read_integers['rpm'].astype(str).astype(float)

In [205]:
df_read_integers['length_linker'] = df_read_integers['end_linker'] - df_read_integers['start_linker']
df_read_integers['length_linker'].value_counts()

4    47960
3    17681
5     2993
2      165
1      148
6        3
Name: length_linker, dtype: int64

In [206]:
df_read_integers= df_read_integers[df_read_integers.length_linker.apply(lambda x: x in [0,1,2,3,4,5,6,7])]
df_read_integers['length_linker'].value_counts()

4    47960
3    17681
5     2993
2      165
1      148
6        3
Name: length_linker, dtype: int64

In [207]:
def check_stop_codon(sequence):
    for aa in sequence:
        if aa == "_":
            return (False)
    return (sequence)

df_read_integers['sequence_read']=df_read_integers.sequence_read.apply(check_stop_codon)
df_read_integers = df_read_integers[df_read_integers.sequence_read != False]
df_read_integers

,sequence_read,rpm,start_linker,end_linker,length_linker
0,QLYH,517013.126727,22,26,4
1,RLR,92253.438863,22,25,3
2,RLR,25353.794140,24,27,3
3,APDGA,22586.924891,22,27,5
4,RLR,15976.162357,20,23,3
...,...,...,...,...,...
98697,QMYH,0.611598,24,28,4
98698,RLPI,0.611598,24,28,4
98699,RLR,0.611598,24,27,3
98700,RLR,0.611598,24,27,3


In [208]:
print(df_read_integers.groupby('sequence_read').size())

sequence_read
A        18
AA        4
AAAD      4
AAAGA     1
AADGA     6
         ..
YTKY     11
YVPR      2
YWGAL     1
YWGPL     3
YWTI      1
Length: 2012, dtype: int64


In [209]:
df_read_integers.groupby('sequence_read')['rpm'].apply(sum)
df_frequency=df_read_integers.groupby('sequence_read')['rpm'].apply(sum)
df_frequency

sequence_read
A        20.794331
AA        2.446392
AAAD      4.281186
AAAGA     0.611598
AADGA     6.727578
           ...    
YTKY      6.727578
YVPR      1.223196
YWGAL     0.611598
YWGPL     1.834794
YWTI      0.611598
Name: rpm, Length: 2012, dtype: float64

In [210]:
with pd.ExcelWriter('Frequency_linkers_'+filename+'.xlsx') as writer:
    df_frequency.to_excel(writer, sheet_name='percentage')